## Implementação de Naive Bayes

O algoritmo de Naive Bayes é baseado em probabilidade condicional. Este é um dos mais simples algoritmos de classificação. Inicialmente, deve-se separar todas as classes, e então para cada _feature_, é necessário encontrar a probabilidade para seus valores.
### Definir as configurações

In [1]:
import numpy as np
import pandas as pd
import math


''' Escolher o dataset que será utilizado '''
df = pd.read_csv("./diabetes.csv")
#df = pd.read_csv("./wine.csv", index_col = False)

''' Escolher a coluna de classe do dataset '''
target = 'Outcome'
#target = 'Alcohol'

''' Obtém as colunas de atributos '''
features = df.columns[df.columns != target]

''' Obtém as classes '''
classes = df[target].unique()

''' Obtém o tamanho do Dataset '''
data_length = len(df.index)

''' Configurações do k-fold
    k: Quantidade de subsets
    num_examples: quantidade, calculada, de exemplos por subset '''
k = 5 
num_examples = math.floor(data_length/k)    

### Construir os Dataset

In [2]:
train = []
test = []

''' Constrói os subsets utilizados para cada iteração do k-fold '''
for i in range(0,k):
    test.append(df[i*num_examples : (i+1)*num_examples])
    train.append(df.drop(test[i].index))

### Cálculo das Probabilidades

Aqui será caluclada as probabilidades de cada classe, e serão armazenadas em uma estrutura de dicionário 

```
dict: 
  keys: classe
  values: dict: 
        keys: atributo
        values: dict:
              keys: valor
              values: probabilidade do valor
```

Deste modo, a probabilidade de cada classe poderá ser facilmente acessada.

In [3]:
probabilities = []
probabilitesPerClass = []

''' Calcula a probabilidade de cada classe, para cada iteração do k-fold'''
for i in range(0,k):
    
    ''' Cria os dicionarios de dados'''
    probabilities.append({})
    probabilitesPerClass.append({})
    
    ''' Itera entre as classes '''
    for x in classes:
        
        ''' Obtém os exemplos somente daquela classe '''
        dfExamplesPerClasses = df[df[target]==x][features]
        
        ''' Total de amostras da classe'''
        samplesPerClass = len(dfExamplesPerClasses)
        
        ''' Vetor de probabilidades (que conterá cada atributo) '''
        classProb = {}
        
        ''' Itera entre os atributos '''
        for col in dfExamplesPerClasses.columns:
            ''' Vetor de probabilidades por'''
            colProb = {}
            
            ''' Itera entre os valores unicos de cada classe '''
            for val, count in dfExamplesPerClasses[col].value_counts().iteritems():
                
                ''' Calcula a probabilidade de um item pertencer àquela classe'''
                prob = count/samplesPerClass
                
                ''' Salva a probabildiade daquela classe '''
                colProb[val] = prob
                
            ''' Salva a probabilidade de um atributo no vetor de probabilidades'''
            classProb[col] = colProb
        
        ''' Salva a quantidade de valores únicos para cada atributo para aquela classe '''
        probabilities[i][x] = classProb
        
        ''' Salva a probabilidade de um elemento pertencer àquela classe '''
        probabilitesPerClass[i][x] = len(dfExamplesPerClasses)/len(df)

        
''' 
Calcula a probabilidade dos elementos de um dataset (de testes)
i -> Iteração do k_Fold
x -> Pandas.Series
 
'''    
def probabs(i, x):    
    
    ''' Argumento deve ser do tipo Serie'''
    if not isinstance(x,pd.Series):
        raise IOError("Argumento deve ser do tipo Pandas.Series")
    
    ''' Cria um vetor vazio de probabilidade '''
    prob = {}
    
    ''' Itera entre as classes do dataset '''
    for cl in classes:
        ''' Verifica a probabilidade de um elemento aleatorio pertencer àquela daquela classe '''
        pr = probabilitesPerClass[i][cl]
        
        ''' Itera entre as colunas e os itens do dataset X '''
        for col,val in x.iteritems():
            ''' Caso o valor daquele elemento seja conhecido, retorna a probabilidade do valor para aquela coluna '''
            try:
                pr *= probabilities[i][cl][col][val]
            except KeyError:
                ''' Caso o valor não seja conhecido, retorna erro '''
                pr = 0
        
        ''' Salva a probabilidade do elemento pertencer aquela classe '''
        prob[cl] = pr
    
    ''' Retorna a probabilidade de cada elemento do dataset pertencer a cada classe '''
    return prob

def classify(i, x):
    # i - Iteração K-Fold
    # x - DataFrame Pandas com índices
    
    ''' Calcula as probabilidades '''
    probab = probabs(i, x)
    
    maxProbability = 0
    maxClass = ''
    
    ''' Verifica a classe com maior probabilidade para cada um dos itens'''
    for curClass, prob in probab.items():
        if prob > maxProbability:
            maxProbability = prob
            maxClass = curClass
            
    return maxClass

### Treinamento

In [4]:
b = []

for l in range(0,k):
    for j in train[l].index:
        # print(classify(l, train[l].loc[j,features]), train[l].loc[j])
        # print('')
        b.append([])
        b[l].append(classify(l, train[l].loc[j,features]) == train[l].loc[j,target])
    
    print(sum(b[l]),"corretos de ",len(train[l]))
    print("Precisão:", sum(b[l])/len(train[l]))
    print('')

602 corretos de  615
Precisão: 0.9788617886178862

602 corretos de  615
Precisão: 0.9788617886178862

602 corretos de  615
Precisão: 0.9788617886178862

602 corretos de  615
Precisão: 0.9788617886178862

603 corretos de  615
Precisão: 0.9804878048780488



### Teste

In [49]:
a = []

for l in range(0,k):
    for j in test[l].index:
        #print(classify(l, test[l].loc[j,features]), test[l].loc[j])
        #print('')
        a.append([])
        a[l].append(classify(l, test[l].loc[j,features]) == test[l].loc[j,target])
    
    print(sum(a[l]),"corretos de",len(test[l]))
    print("Precisão:", sum(a[l])/len(test[l]))
    print('')

150 corretos de 153
Precisão: 0.9803921568627451

150 corretos de 153
Precisão: 0.9803921568627451

150 corretos de 153
Precisão: 0.9803921568627451

150 corretos de 153
Precisão: 0.9803921568627451

149 corretos de 153
Precisão: 0.9738562091503268

